In [7]:
# import statements:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
# run this just in case
nltk.download('all')
nltk.download('stopwords')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/rohitkandala/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/rohitkandala/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/rohitkandala/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/rohitkandala/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/rohitkandala/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_

True

In [9]:
# reading in csv as dataframe:
yelp_data = pd.read_csv("yelp_true_sample_100k.csv")

In [10]:
# applying Summer's preprocess_text() function:
def preprocess_text(text):
    # tokenizing test, ensuring it is not case insensitive
    tokens = nltk.word_tokenize(text.lower())
    
    # removing stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # joining filtered tokens back into a string
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text

# apply function to 'text' column in yelp_data
yelp_data['text'] = yelp_data['text'].apply(preprocess_text)

In [12]:
# binary classification for yelp data based on star: 
yelp_data['sentiment'] = yelp_data['stars'].apply(lambda x: 0 if x < 3 else 1)

In [13]:
yelp_data

,Unnamed: 0,stars,text,date,sentiment
0,0,1,ordered online curbside pickup.i informed 30mi...,2021-09-26 14:29:01,0
1,1,3,place really nice staffs food got n't best.the...,2016-03-25 23:09:51,1
2,2,5,grilled oyster def best french quarter . tried...,2019-12-28 20:17:36,1
3,3,4,started going place 6 months ago 's become apa...,2014-10-25 20:47:28,1
4,4,4,discovered foghorn sunday evening searching on...,2016-10-10 19:23:47,1
...,...,...,...,...,...
99995,99995,5,"& l awesome . months , `` '' , rattling fridge...",2019-04-17 13:08:41,1
99996,99996,5,steinbeck wonderful caring veterinarian practi...,2018-02-10 04:21:45,1
99997,99997,5,great restaurant ! 's modest restaurant nice p...,2021-10-03 10:12:12,1
99998,99998,2,stopped ihop dropping dog twb2 . closest locat...,2012-08-05 00:27:42,0
